In [ ]:
#@title Import relevant modules
import os
import sys
import pandas as pd
import geopandas as gpd
import sqlalchemy as sq
from matplotlib import pyplot as plt
from dotenv import load_dotenv
from DataService import DataService
from ClimateDataRequester import ClimateDataRequester as cdr

In [ ]:
# The following lines adjust the granularity of reporting.
pd.options.display.max_rows = 100
pd.options.display.float_format = "{:.1f}".format
pd.set_option('display.max_columns', None)


In [ ]:
os.chdir('/data')
load_dotenv('docker/.env')
PGUSER = os.getenv('POSTGRES_USER')
PGPW = os.getenv('POSTGRES_PW')
PGDB = os.getenv('POSTGRES_DB')

In [ ]:
def push_data(df: pd.DataFrame) -> None:
    dataService = DataService(PGDB, PGUSER, PGPW)
    db_con = dataService.connect()
    df.to_sql("WeatherData", db_con, if_exists="append", index=False)
    dataService.cleanup()
